# Self-Query

`SelfQueryRetriever` 는 자체적으로 질문을 생성하고 해결할 수 있는 기능을 갖춘 검색 도구입니다.

이는 사용자가 제공한 자연어 질의를 바탕으로, query-constructing LLM chain을 사용해 구조화된 질의를 만듭니다. 그 후, 이 구조화된 질의를 기본 벡터 데이터 저장소(VectorStore)에 적용하여 검색을 수행합니다.

이 과정을 통해, SelfQueryRetriever 는 단순히 사용자의 입력 질의를 저장된 문서의 내용과 의미적으로 비교하는 것을 넘어서, 사용자의 질의에서 문서의 메타데이터에 대한 필터를 추출 하고, 이 필터를 실행하여 관련된 문서를 찾을 수 있습니다.


:테이블 구조로 되어있는 데이터에 self-query retriever에 적합하-다

- 자체적으로 질문을 생성하고 해결할 수 있는 기능. 자체적으로 query를 만들고 db에 질문을 던진다.
- 테이블 구조로 되어있는데 이 안에 데이터가 들어가 있는경우. 엑셀안에 자연어만 있으면 크게 의미가 없는데, 년도, 작성자, 상품정보, 카테고리 등등을 필터링해서 사용해야 할때 유용하다.

- 모든 db가 지원하지 않는다. faiss는 되지 않음!


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import CSVLoader

In [5]:
loader = CSVLoader("./data/titanic.csv")
docs = loader.load()

In [11]:
docs[1].metadata

{'source': './data/titanic.csv', 'row': 1}

In [13]:
print(docs[1].page_content)

PassengerId: 2
Survived: 1
Pclass: 1
Name: Cumings, Mrs. John Bradley (Florence Briggs Thayer)
Sex: female
Age: 38
SibSp: 1
Parch: 0
Ticket: PC 17599
Fare: 71.2833
Cabin: C85
Embarked: C


In [8]:
vector_store=Chroma.from_documents(
    docs, OpenAIEmbeddings=(model="text-embedding-3-small")
)

In [14]:
loader = CSVLoader(
    file_path = "./data/titanic.csv",
    csv_args={
        "delimiter": ',', # 구분자. tab인 경우에는 tab으로 바꿔줌
        "quotechar": '"', # 인용 부호 문자
        "fieldnames": [
            "PassengerId",
            "Survived (1: Survived, 0: Died)",
            "Passenger Class",
            "Name",
            "Sex",
            "Age",
            "Number of Siblings/Spouses Aboard",
            "Number of Parents/ Children Aboard",
            "Ticken Number",
            "Fare",
            "Cabin",
            "Port of Embarkation",
        ], #필드이름
    }
)
docs = loader.load()

In [21]:
print(docs[10].page_content)

PassengerId: 10
Survived (1: Survived, 0: Died): 1
Passenger Class: 2
Name: Nasser, Mrs. Nicholas (Adele Achem)
Sex: female
Age: 14
Number of Siblings/Spouses Aboard: 1
Number of Parents/ Children Aboard: 0
Ticken Number: 237736
Fare: 30.0708
Cabin: 
Port of Embarkation: C


In [22]:
# SelfQueryEmbedding

from langchain.chains.query_constructor.base import AttributeInfo


# 메타데이터 필드 정보 생성
metadata_field_info = [
    AttributeInfo(
        name="PassengerId",
        description="고객 id",
        type="integer",
    ),
    AttributeInfo(
        name="Survived ",
        description="생존 여부. (1일경경우 생존, 0일경우 사망)",
        type="integer",
    ),
    AttributeInfo(
        name="Passenger Class",
        description="좌석 등급",
        type="integer",
    ),
    AttributeInfo(
        name="Name",
        description="고객 이름",
        type="string",
    ),
    AttributeInfo(
        name="Age",
        description="고객의 나이",
        type="integer",
    ),
    AttributeInfo(
        name="Number of Siblings/Spouses Aboard",
        description="탑승 형제/ 배우자의 수",
        type="integer",
    ),
    AttributeInfo(
        name="Ticken Number",
        description="티켓 넘버",
        type="integer",
    ),
    AttributeInfo(
        name="Fare",
        description="요금",
        type="integer",
    ),
    AttributeInfo(
        name="Cabin",
        description="객식",
        type="integer",
    ),
    AttributeInfo(
        name="Port of Embarkation",
        description="탑승 항구",
        type="string",
    ),
]



In [25]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vector_store,
    document_contents="타이타닉 탑승 고객 정보",
    metadata_field_info=metadata_field_info
)

In [27]:
retriever.invoke("PassengerID가 1인 고객의 정보에 대해서 알려주세요")

[]